In [23]:
# Import libraries
import numpy as np
import pandas as pd

import os

#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge

import string

import datetime

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Constants goes here
NMONTHS = 24
pd.set_option('max_columns', 400)

In [3]:
# Import data
DATA_FOLDER = 'Data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [4]:
# For decrease memory consumption
def downcast_dtypes(df):
    '''
    Changes column types in the dataframe: 

            `float64` type to `float32`
            `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [5]:
transactions = downcast_dtypes(transactions)
items = downcast_dtypes(items)
item_categories = downcast_dtypes(item_categories)
shops = downcast_dtypes(shops)
test = downcast_dtypes(test)

In [6]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
# New function to construct base dataframes for train, validation and test
def construct_base_train_val_test(months=NMONTHS):
    """
    months - number of last months to construct base train/test matrix dataset
    This function construct grouped by months sales information for train, validation and test
    """
    # Group transactions by month, shop and item (add price mean here)
    grp_trans = transactions.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'item_cnt_day': 'sum'})
    
    # Split dataset by months
    lst_grp_trans = []
    max_month = grp_trans.date_block_num.max() + 1
    for month in range(max_month-months-2, max_month): # Get only last year data
        month_df = grp_trans[grp_trans.date_block_num == month]
        # Rename target column and price column
        month_df.rename(index=str, columns={"item_cnt_day": "item_cnt_day_" + str(month)}, inplace=True) 
        lst_grp_trans.append(month_df)
        
        
    # Join data with test dataset
    all_data = test[['shop_id', 'item_id']].set_index(['shop_id', 'item_id'])
    for month in range(max_month-months-2, max_month): # [8..34)
        all_data = all_data.join(lst_grp_trans[month+months-max_month+2][['shop_id', 'item_id', 'item_cnt_day_'+str(month)]
                                                                        ].set_index(['shop_id', 'item_id']))
        
    #print(all_data.head())
    
    # Replace NaNs by 0
    all_data.fillna(0, inplace=True)
    
    # Get train target and validation target column
    val_target = all_data.item_cnt_day_33
    train_target = all_data.item_cnt_day_32
    
    # Calculate x_train, x_val, x_test
    x_train = all_data.copy()
    x_train.drop(['item_cnt_day_32', 'item_cnt_day_33'], axis=1, inplace=True)
    
    x_val = all_data.copy()
    first_col = str(max_month-months-2)
    x_val.drop(['item_cnt_day_' + first_col, 'item_cnt_day_33',], 
               axis=1, inplace=True)
    
    x_test = all_data.copy()
    second_col = str(max_month-months-1)
    x_test.drop(['item_cnt_day_' + first_col, 'item_cnt_day_' + second_col], axis=1, inplace=True)
    
    return x_train, x_val, x_test, train_target, val_target

In [8]:
%%time
tr_train, tr_val, tr_test, tr_target, val_target = construct_base_train_val_test()
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 24) (214200, 24) (214200, 24)
Wall time: 9.08 s


In [9]:
# TF-IDF transformation to the item names
tfidf = TfidfVectorizer()
item_name_matrix = tfidf.fit_transform(items.item_name)
item_name_matrix.shape
# (22170, 18222)

(22170, 18222)

In [10]:
%%time
tsvd = TruncatedSVD(n_components=300, random_state=26)
tsvd_item_matrix = tsvd.fit_transform(item_name_matrix)

Wall time: 6.43 s


In [11]:
# Join with the items dataframe
items = items.join(pd.DataFrame(tsvd_item_matrix, index=items.index))
items.head()

,item_name,item_id,item_category_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.000641,0.000150,6.665875e-04,-0.000096,0.000303,-0.000537,-0.000260,-0.000361,1.628417e-03,0.000099,0.000056,0.000430,0.001018,0.001085,-0.000322,0.000328,0.000130,0.000627,-0.000143,-0.000688,0.000116,0.000966,0.000732,0.000976,0.000311,-0.000096,-0.000031,0.000353,-0.000332,0.000085,-0.000701,0.002100,-0.000286,0.000899,0.000522,0.000392,-0.000219,1.390099e-03,-0.001162,-0.000480,0.000434,0.000671,-0.000319,0.001602,0.000208,-0.000112,0.000825,-0.000135,0.000046,-0.000130,-0.000582,-0.000442,-0.001379,0.000992,0.000030,0.000997,0.000102,1.513968e-03,-0.000998,-0.000825,-0.001091,-0.000240,0.002661,-0.000658,0.001752,-0.000320,-0.002273,-0.001298,-0.000505,-0.000826,-0.000964,-0.002076,0.000972,0.001193,0.000881,-0.001403,0.000341,0.000471,0.000686,-0.000117,0.001055,0.002024,-0.001867,0.000093,0.000649,-0.000272,-0.001606,-0.000111,-0.000499,-0.001416,-0.000739,0.000289,-0.002100,0.001139,-0.000337,-0.000141,-0.000469,-0.000754,0.002121,0.001653,0.002613,0.001338,-0.002171,-0.002578,0.002437,-0.002180,-0.000174,-0.001672,-0.000575,-0.000135,0.001852,0.000132,0.002096,0.000193,-0.001350,0.001015,0.001434,0.000269,-0.000181,-0.002386,0.002269,-0.001062,0.001876,-0.000378,0.000191,0.000332,0.000331,-0.001211,0.002408,0.001586,-0.000629,0.001917,0.002461,-0.001492,-0.002915,0.001704,-0.002238,-0.001447,0.000138,0.002142,0.001178,0.000871,0.001367,0.002531,0.000747,0.000007,0.001785,0.002238,0.000483,-0.000676,0.000595,0.001470,0.001023,0.000780,-0.002417,0.000295,-0.000854,0.003376,-0.001158,0.002055,-0.002853,0.000794,0.000801,-0.002082,0.002438,0.000499,0.003421,0.000175,-0.002071,-0.001182,-0.000732,0.001006,-0.000963,-0.000943,0.001838,-0.002317,-0.000641,-0.004470,-0.002378,-0.000183,-0.000070,-0.002607,-0.005434,0.000276,0.005320,-0.001488,0.000495,0.004292,-0.001266,-0.000966,0.002701,-0.007133,-0.003252,-0.001798,0.005546,-0.002238,-0.003755,0.006275,0.001321,-0.001671,0.004613,-0.004375,-0.008932,0.000599,-0.002665,0.003900,-0.002043,0.002998,0.006054,-0.001779,0.004422,0.000089,-0.001420,0.002192,0.004981,0.000067,0.001476,-0.004590,-0.000585,0.000607,-0.001207,0.001108,-0.000548,0.001221,-0.001700,-0.000938,-0.002201,0.002105,-0.000432,-0.001797,0.000828,-0.000966,0.001502,-0.002130,-0.000082,-0.002346,-0.000823,-0.000978,-0.002553,-0.000610,0.000022,0.000980,0.002423,0.000793,-0.000691,-0.000927,-0.000184,0.002126,0.000368,0.000145,-0.000311,-0.001380,-0.000970,0.002174,-0.000228,-0.000024,-0.000513,0.000714,-0.000423,-0.001445,0.000936,0.000292,0.001015,-0.002972,-0.004625,-0.003633,-0.000317,-0.001973,0.000128,0.000077,-0.001764,-0.001665,-0.002880,-0.000664,0.001539,-0.003229,0.002571,0.000372,-0.002971,-0.002640,0.001137,0.000090,-0.000583,-0.000975,0.000037,0.001724,-0.003010,0.001215,-0.001037,0.000481,0.002997,0.001726,0.002455,-0.001184,-0.000464,0.001044,-0.0022

In [12]:
# Add means of item_cnt_day
def add_means(train, val, test, months=NMONTHS):
    """
    train - base train dataframe
    val - base validation dataframe (train, shifted by 1 month further)
    test - base test dataframe (val, shifted by 1 month further)
    This function add sliding means by last n column values (last n months)
    """
    # Define filters for price and item count features separation
    train_filter_item_cnt_col = [col for col in train if col.startswith('item_cnt_day')]
    val_filter_item_cnt_col = [col for col in val if col.startswith('item_cnt_day')]
    test_filter_item_cnt_col = [col for col in test if col.startswith('item_cnt_day')]
    
    # Get 6 separate dataframes
    train_item_cnt_df = train[train_filter_item_cnt_col]
    val_item_cnt_df = val[val_filter_item_cnt_col]
    test_item_cnt_df = test[test_filter_item_cnt_col]
    
    # Add features to train
    np_train_count = np.array(train_item_cnt_df)
    
    for i in range(1, months+1):
        train['mean_cnt_' + str(i)] = pd.Series(np_train_count[:, -i:].mean(axis=1), index=train.index)
        
    # Add features to validation
    np_val_count = np.array(val_item_cnt_df)
    
    for i in range(1, months+1):
        val['mean_cnt_' + str(i)] = pd.Series(np_val_count[:, -i:].mean(axis=1), index=train.index)
        
    # Add features to test
    np_test_count = np.array(test_item_cnt_df)
    
    for i in range(1, months+1):
        test['mean_cnt_' + str(i)] = pd.Series(np_test_count[:, -i:].mean(axis=1), index=test.index)
        
    return train, val, test

In [13]:
%%time
tr_train, tr_val, tr_test = add_means(tr_train, tr_val, tr_test)
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 48) (214200, 48) (214200, 48)
Wall time: 429 ms


In [14]:
# Add some mean encoded features
def get_month_mean_encodings(train, val, test):
    """
    train - base train dataframe
    val - base validation dataframe (train, shifted by 1 month further)
    test - base test dataframe (val, shifted by 1 month further)
    Function calculate cumsum divided by cumcount for current shop and current item saled quantities,
    also it makes the same transformations for current month price
    We aggregate info by months
    """
    # Drop indexes
    train.reset_index(inplace=True)
    val.reset_index(inplace=True)
    test.reset_index(inplace=True)
    
    # Get dataframes with sold item quantities, grouped by shop and item
    
    # Define item count column filters
    train_filter_cnt_col = [col for col in train if col.startswith('item_cnt_day')]
    val_filter_cnt_col = [col for col in val if col.startswith('item_cnt_day')]
    test_filter_cnt_col = [col for col in test if col.startswith('item_cnt_day')]
    
    # Aggregate train by shop and item
    train_shop_agg_cnt_df = train.groupby(['shop_id'])[train_filter_cnt_col].agg(['sum'])
    train_item_agg_cnt_df = train.groupby(['item_id'])[train_filter_cnt_col].agg(['sum'])
    
    # Aggregate validation set by shop and item
    val_shop_agg_cnt_df = val.groupby(['shop_id'])[val_filter_cnt_col].agg(['sum'])
    val_item_agg_cnt_df = val.groupby(['item_id'])[val_filter_cnt_col].agg(['sum'])
    
    # Aggregate test set by shop and item
    test_shop_agg_cnt_df = test.groupby(['shop_id'])[test_filter_cnt_col].agg(['sum'])
    test_item_agg_cnt_df = test.groupby(['item_id'])[test_filter_cnt_col].agg(['sum'])
    

    # Now we get train cumulative sum of this features on rows
    train_shop_agg_cnt_df = train_shop_agg_cnt_df.cumsum(axis=1)
    train_item_agg_cnt_df = train_item_agg_cnt_df.cumsum(axis=1)
    
    # Get validation cumulative sum of this features on rows
    val_shop_agg_cnt_df = val_shop_agg_cnt_df.cumsum(axis=1)
    val_item_agg_cnt_df = val_item_agg_cnt_df.cumsum(axis=1)

    # Get test cumulative sum of this features on rows
    test_shop_agg_cnt_df = test_shop_agg_cnt_df.cumsum(axis=1)
    test_item_agg_cnt_df = test_item_agg_cnt_df.cumsum(axis=1)

    # Calculate arithmetic progression (1, 2, 3, 4, ...) with months length
    np_to_divide = np.array(train_item_agg_cnt_df.columns.labels[0] + 1, dtype=float)
    
    # Divide train cumulative sum by column number (np.cumcount emulation)
    train_shop_agg_cnt_df = train_shop_agg_cnt_df / np_to_divide
    train_item_agg_cnt_df = train_item_agg_cnt_df / np_to_divide
    
    # Divide validation cumulative sum by column number (np.cumcount emulation)
    val_shop_agg_cnt_df = val_shop_agg_cnt_df / np_to_divide
    val_item_agg_cnt_df = val_item_agg_cnt_df / np_to_divide
    
    # Divide test cumulative sum by column number (np.cumcount emulation)
    test_shop_agg_cnt_df = test_shop_agg_cnt_df / np_to_divide
    test_item_agg_cnt_df = test_item_agg_cnt_df / np_to_divide
    
    
    # Move index to columns of train daraframes
    train_shop_agg_cnt_df.reset_index(inplace=True)
    train_item_agg_cnt_df.reset_index(inplace=True)
    
    # Move index to columns of validation daraframes
    val_shop_agg_cnt_df.reset_index(inplace=True)
    val_item_agg_cnt_df.reset_index(inplace=True)
    
    # Move index to columns of test daraframes
    test_shop_agg_cnt_df.reset_index(inplace=True)
    test_item_agg_cnt_df.reset_index(inplace=True)

    
    # Join all train mean encoding features with initial data
    train_ext = pd.merge(train, train_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    train_ext = pd.merge(train_ext, train_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')

    # Join all validation mean encoding features with initial data
    val_ext = pd.merge(val, val_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    val_ext = pd.merge(val_ext, val_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')
                       
    # Join all test mean encoding features with initial data
    test_ext = pd.merge(test, test_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    test_ext = pd.merge(test_ext, test_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')

    return train_ext, val_ext, test_ext

In [15]:
%%time
tr_train, tr_val, tr_test = get_month_mean_encodings(tr_train, tr_val, tr_test)

# Drop duplicate column
tr_train.drop(['item_id_x'], axis=1, inplace=True)
tr_val.drop(['item_id_x'], axis=1, inplace=True)
tr_test.drop(['item_id_x'], axis=1, inplace=True)

print(tr_train.shape, tr_val.shape, tr_test.shape)

KeyError: "labels ['item_id_x'] not contained in axis"

In [16]:
tr_train.head()

,shop_id,item_id,item_cnt_day_8_x,item_cnt_day_9_x,item_cnt_day_10_x,item_cnt_day_11_x,item_cnt_day_12_x,item_cnt_day_13_x,item_cnt_day_14_x,item_cnt_day_15_x,item_cnt_day_16_x,item_cnt_day_17_x,item_cnt_day_18_x,item_cnt_day_19_x,item_cnt_day_20_x,item_cnt_day_21_x,item_cnt_day_22_x,item_cnt_day_23_x,item_cnt_day_24_x,item_cnt_day_25_x,item_cnt_day_26_x,item_cnt_day_27_x,item_cnt_day_28_x,item_cnt_day_29_x,item_cnt_day_30_x,item_cnt_day_31_x,mean_cnt_1,mean_cnt_2,mean_cnt_3,mean_cnt_4,mean_cnt_5,mean_cnt_6,mean_cnt_7,mean_cnt_8,mean_cnt_9,mean_cnt_10,mean_cnt_11,mean_cnt_12,mean_cnt_13,mean_cnt_14,mean_cnt_15,mean_cnt_16,mean_cnt_17,mean_cnt_18,mean_cnt_19,mean_cnt_20,mean_cnt_21,mean_cnt_22,mean_cnt_23,mean_cnt_24,"('item_cnt_day_8', 'sum')_x","('item_cnt_day_9', 'sum')_x","('item_cnt_day_10', 'sum')_x","('item_cnt_day_11', 'sum')_x","('item_cnt_day_12', 'sum')_x","('item_cnt_day_13', 'sum')_x","('item_cnt_day_14', 'sum')_x","('item_cnt_day_15', 'sum')_x","('item_cnt_day_16', 'sum')_x","('item_cnt_day_17', 'sum')_x","('item_cnt_day_18', 'sum')_x","('item_cnt_day_19', 'sum')_x","('item_cnt_day_20', 'sum')_x","('item_cnt_day_21', 'sum')_x","('item_cnt_day_22', 'sum')_x","('item_cnt_day_23', 'sum')_x","('item_cnt_day_24', 'sum')_x","('item_cnt_day_25', 'sum')_x","('item_cnt_day_26', 'sum')_x","('item_cnt_day_27', 'sum')_x","('item_cnt_day_28', 'sum')_x","('item_cnt_day_29', 'sum')_x","('item_cnt_day_30', 'sum')_x","('item_cnt_day_31', 'sum')_x","(item_cnt_day_8_y, sum)","(item_cnt_day_9_y, sum)","(item_cnt_day_10_y, sum)","(item_cnt_day_11_y, sum)","(item_cnt_day_12_y, sum)","(item_cnt_day_13_y, sum)","(item_cnt_day_14_y, sum)","(item_cnt_day_15_y, sum)","(item_cnt_day_16_y, sum)","(item_cnt_day_17_y, sum)","(item_cnt_day_18_y, sum)","(item_cnt_day_19_y, sum)","(item_cnt_day_20_y, sum)","(item_cnt_day_21_y, sum)","(item_cnt_day_22_y, sum)","(item_cnt_day_23_y, sum)","(item_cnt_day_24_y, sum)","(item_cnt_day_25_y, sum)","(item_cnt_day_26_y, sum)","(item_cnt_day_27_y, sum)","(item_cnt_day_28_y, sum)","(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)"
0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,3.0,2.0,1.666667,1.50,1.2,1.000000,0.857143,1.000,1.111111,1.1,1.000000,1.000000,0.923077,0.857143,0.800000,0.7500,0.705882,0.666667,0.631579,0.60,0.571429,0.545455,0.521739,0.500000,446.0,469.0,504.666656,612.75,636.599976,632.666687,653.428589,641.875,645.555542,654.400024,675.0,701.916687,713.692322,722.071411,743.266663,796.3125,812.588257,813.944458,817.473694,821.049988,824.523804,825.954529,827.956543,844.208313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.076923,16.714285,19.133333,30.4375,33.529411,33.444443,33.157894,32.75,35.095238,37.818180,38.478260,41.750000
1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,446.0,469.0,504.666656,612.75,636.599976,632.666687,653.428589,641.875,645.555542,654.400024,675.0,701.916687,713.692322,722.071411,743.266663,796.3125,812.588257,813.944458,817.473694,821.049988,824.523804,825.954529,827.956543,844.208313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,1.0,0.5,1.000000,1.50,1.2,1.000000,0.857143,0.750,0.666667,0.6,0.545455,0.500000,0.461538,0.428571,0.400000,0.3750,0.352941,0.333333,0.315789,0.30,0.285714,0.272727,0.260870,0.250000,446.0,469.0,504.666656,612.75,636.599976,632.666687,653.428589,641.875,645.555542,654.400024,675.0,701.916687,713.692322,722.071411,743.266663,796.3125,812.588257,813.944458,817.473694,821.049988,824.523804,825.954529,827.956543,8

In [17]:
# Extract city column from shops dataframe
shop_name_split = [el.split(' ') for el in shops.shop_name.values]
shops['city'] = pd.Series([el[0] for el in shop_name_split], index=shops.index)
shops.head(10)

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский
5,"Вологда ТРЦ ""Мармелад""",5,Вологда
6,"Воронеж (Плехановская, 13)",6,Воронеж
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж
9,Выездная Торговля,9,Выездная


In [18]:
# Add city feature
tr_train = pd.merge(tr_train, shops, how='left', left_on='shop_id', right_on='shop_id')
tr_val = pd.merge(tr_val, shops, how='left', left_on='shop_id', right_on='shop_id')
tr_test = pd.merge(tr_test, shops, how='left', left_on='shop_id', right_on='shop_id')
print(tr_train.shape, tr_val.shape, tr_test.shape)

# Drop unnecessary shop_name column
tr_train.drop(['shop_name'], axis=1, inplace=True)
tr_val.drop(['shop_name'], axis=1, inplace=True)
tr_test.drop(['shop_name'], axis=1, inplace=True)
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 100) (214200, 100) (214200, 100)
(214200, 99) (214200, 99) (214200, 99)


In [19]:
# Factorize city name
tr_train['city'] = tr_train['city'].factorize()[0]
tr_val['city'] = tr_val['city'].factorize()[0]
tr_test['city'] = tr_test['city'].factorize()[0]

In [20]:
# Add tf-idf item name features
tr_train = tr_train.join(items, how='left', on='item_id', rsuffix='_itm')
tr_val = tr_val.join(items, how='left', on='item_id', rsuffix='_itm')
tr_test = tr_test.join(items, how='left', on='item_id', rsuffix='_itm')

# Drop useless columns
tr_train.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)
tr_val.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)
tr_test.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)

print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 400) (214200, 400) (214200, 400)


In [21]:
tr_train.iloc[:, -305:].head()

,"(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)",city,item_category_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,37.818180,38.478260,41.750000,0,19,0.079082,0.002221,0.020591,0.190210,0.019453,0.285132,-0.223680,0.280355,0.210752,-0.050954,0.077036,0.080032,-0.163381,0.015022,0.112685,-0.061891,-0.049274,-0.045630,-0.017297,-0.021274,0.027726,0.019591,-0.019144,0.017491,-0.043102,0.006013,-0.015368,-0.023692,0.003968,-0.022937,0.001272,-0.004655,-0.035973,-0.054491,-0.022236,-0.033944,-0.022346,-0.006857,0.005182,-0.012873,0.021376,-0.017513,0.011230,-0.013015,-0.015055,-0.102259,0.022022,-0.040760,0.002774,-0.019000,-0.025949,-0.022804,0.005145,-0.026562,-0.013219,0.014977,0.011392,0.011584,0.021308,-0.000551,0.012479,0.015555,0.019622,0.000181,0.005953,-0.003875,-0.014303,0.020001,-0.063006,0.022029,-0.017804,-0.004072,-0.005042,-0.013528,-0.008939,0.043461,-0.043720,-0.022917,0.023542,-0.030945,-0.003825,0.010244,-0.021132,0.023140,-0.022852,0.017399,0.005681,-0.028971,-0.027390,-0.006911,0.021308,0.009292,-0.017350,-0.000147,-0.003343,-0.006062,0.009291,0.005816,-0.041124,0.015491,0.001717,-0.029106,-0.019643,-0.003939,0.016604,-0.002345,-0.002582,-0.043976,0.019029,-0.003705,-0.006136,-0.005814,-0.015199,0.013490,-0.021743,0.010248,-0.004710,-0.010066,0.004367,-0.016443,0.009904,-0.014784,0.013932,-0.009924,0.007120,-0.004698,0.010167,-0.015185,0.039849,0.022425,0.006819,0.021993,-0.038858,-0.015359,0.032628,-0.005043,-0.004266,-0.021465,-0.006647,-0.017699,-0.014367,0.006524,0.005086,0.024749,0.009834,0.012348,-0.018169,-0.006818,-0.014469,-0.007364,-0.025772,0.004186,0.016532,-0.024144,0.009763,0.002107,0.024014,0.015460,0.037792,0.031135,-0.036164,0.037063,0.015953,-0.012918,0.002167,0.006488,0.029965,-0.041069,0.006577,-0.041852,0.004722,0.003370,0.026239,0.009073,0.031113,-0.037213,-0.017332,0.040085,0.037619,-0.040824,0.010660,-0.027082,-0.006413,-0.012178,0.041732,-0.022731,-0.013488,-0.025942,-0.006729,-0.070825,-0.009103,-0.006033,0.007976,-0.005644,-0.003217,0.010186,-0.014129,-0.015376,-0.035695,0.040684,-0.050095,0.024948,-0.053320,-0.042195,0.028610,-0.002905,-0.039563,-0.006819,0.003650,0.003939,0.012286,-0.027344,0.027665,-0.005295,-0.011571,0.020388,-0.005933,0.008467,-0.035216,0.026737,-0.030921,-0.024796,-0.033065,-0.068224,0.016837,-0.038565,0.036673,-0.008856,-0.043477,0.006980,0.010261,0.048092,-0.030301,0.012408,-0.079709,0.035322,0.001409,-0.000025,-0.006253,-0.007560,-0.029462,0.034594,0.034168,0.042490,0.016874,0.140981,0.023657,-0.025401,0.016997,0.052940,0.036615,0.003027,0.020136,0.003438,0.038340,0.045190,-0.013041,0.008444,0.119370,-0.010550,-0.010067,-0.006573,0.068689,0.036465,-0.010815,0.009040,0.014500,0.034400,0.050978,0.039984,0.037195,0.037316,0.047825,-0.067076,-0.019021,-0.003786,0.061849,-0.022502,-0.041246,0.031843,0.011609,0.001150,0.023896,0.049366,-0.047553,0.060254,0.046260,-0.025228,-0.011181,0.008685,0.017789,-0.0

In [24]:
# Prepare and scale data
np_train = np.array(tr_train)
np_val = np.array(tr_val)
np_test = np.array(tr_test)

tr_target_clip = np.clip(np.array(tr_target), 0, 20)
val_target_clip = np.clip(np.array(val_target), 0, 20)

In [33]:
tr_train.describe()

,shop_id,item_id,item_cnt_day_8_x,item_cnt_day_9_x,item_cnt_day_10_x,item_cnt_day_11_x,item_cnt_day_12_x,item_cnt_day_13_x,item_cnt_day_14_x,item_cnt_day_15_x,item_cnt_day_16_x,item_cnt_day_17_x,item_cnt_day_18_x,item_cnt_day_19_x,item_cnt_day_20_x,item_cnt_day_21_x,item_cnt_day_22_x,item_cnt_day_23_x,item_cnt_day_24_x,item_cnt_day_25_x,item_cnt_day_26_x,item_cnt_day_27_x,item_cnt_day_28_x,item_cnt_day_29_x,item_cnt_day_30_x,item_cnt_day_31_x,mean_cnt_1,mean_cnt_2,mean_cnt_3,mean_cnt_4,mean_cnt_5,mean_cnt_6,mean_cnt_7,mean_cnt_8,mean_cnt_9,mean_cnt_10,mean_cnt_11,mean_cnt_12,mean_cnt_13,mean_cnt_14,mean_cnt_15,mean_cnt_16,mean_cnt_17,mean_cnt_18,mean_cnt_19,mean_cnt_20,mean_cnt_21,mean_cnt_22,mean_cnt_23,mean_cnt_24,"('item_cnt_day_8', 'sum')_x","('item_cnt_day_9', 'sum')_x","('item_cnt_day_10', 'sum')_x","('item_cnt_day_11', 'sum')_x","('item_cnt_day_12', 'sum')_x","('item_cnt_day_13', 'sum')_x","('item_cnt_day_14', 'sum')_x","('item_cnt_day_15', 'sum')_x","('item_cnt_day_16', 'sum')_x","('item_cnt_day_17', 'sum')_x","('item_cnt_day_18', 'sum')_x","('item_cnt_day_19', 'sum')_x","('item_cnt_day_20', 'sum')_x","('item_cnt_day_21', 'sum')_x","('item_cnt_day_22', 'sum')_x","('item_cnt_day_23', 'sum')_x","('item_cnt_day_24', 'sum')_x","('item_cnt_day_25', 'sum')_x","('item_cnt_day_26', 'sum')_x","('item_cnt_day_27', 'sum')_x","('item_cnt_day_28', 'sum')_x","('item_cnt_day_29', 'sum')_x","('item_cnt_day_30', 'sum')_x","('item_cnt_day_31', 'sum')_x","(item_cnt_day_8_y, sum)","(item_cnt_day_9_y, sum)","(item_cnt_day_10_y, sum)","(item_cnt_day_11_y, sum)","(item_cnt_day_12_y, sum)","(item_cnt_day_13_y, sum)","(item_cnt_day_14_y, sum)","(item_cnt_day_15_y, sum)","(item_cnt_day_16_y, sum)","(item_cnt_day_17_y, sum)","(item_cnt_day_18_y, sum)","(item_cnt_day_19_y, sum)","(item_cnt_day_20_y, sum)","(item_cnt_day_21_y, sum)","(item_cnt_day_22_y, sum)","(item_cnt_day_23_y, sum)","(item_cnt_day_24_y, sum)","(item_cnt_day_25_y, sum)","(item_cnt_day_26_y, sum)","(item_cnt_day_27_y, sum)","(item_cnt_day_28_y, sum)","(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)",city,item_category_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
count,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.00000

In [34]:
# Apply log transformation to the arrays
np_train[:,:100] = np.log1p(np_train[:,:100])
np_val[:,:100] = np.log1p(np_val[:,:100])
np_test[:,:100] = np.log1p(np_test[:,:100])

In [43]:
# Convert nans to numbers
np_train = np.nan_to_num(np_train)
np_val = np.nan_to_num(np_val)
np_test = np.nan_to_num(np_test)

In [49]:
pd.DataFrame(np_train).to_csv('np_train.csv')

In [50]:
np_train.shape

(214200, 400)

In [51]:
# Scale feature matrices
scaler = StandardScaler()
scaler.fit(np_train)

np_train = scaler.transform(np_train)
np_val = scaler.transform(np_val)
np_test = scaler.transform(np_test)

In [52]:
np_train[:5, :]

array([[-1.83583904, -0.54229122,         nan, ...,  2.74524398,
        -2.67678114,  0.48958367],
       [-1.83583904, -0.4833661 ,         nan, ..., -0.24552048,
         0.32794862, -0.37943567],
       [-1.83583904, -0.50114049,         nan, ..., -0.02828303,
         0.73543028, -0.62988265],
       [-1.83583904, -0.5013465 ,         nan, ..., -0.62564011,
        -0.30898175, -1.25221431],
       [-1.83583904, -0.49395461,         nan, ...,  0.1913787 ,
         0.41013463, -0.29602646]])